### Import section

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from skimage import color
from skimage.feature import hog

In [3]:
import warnings
import itertools
from PIL import Image
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
# !pip install visualkeras
import tensorflow as tf
import visualkeras
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.model_selection   import train_test_split
from keras.preprocessing.image import ImageDataGenerator

warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
# # General parameters
# np.random.seed(42)
# tf.random.set_seed(42)

In [6]:
# pip install opencv-python
import cv2
import os

folder_path = "ds_two"             # base/root directory

no_tumor = os.listdir(folder_path + '/no_tumor/')
glioma = os.listdir(folder_path + '/glioma/')
meningioma = os.listdir(folder_path + '/meningioma/')
pituitary = os.listdir(folder_path + '/pituitary/')
craniopharyngioma = os.listdir(folder_path + '/craniopharyngioma_final_1070/')

# no_tumor, glioma, meningioma, pituitary, craniopharyngioma

dataset=[]
lab=[]

In [7]:
for image_name in no_tumor:
    image=cv2.imread(folder_path + '/no_tumor/' + image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((240,240))
    dataset.append(np.array(image))
    lab.append(0)

In [8]:
for image_name in glioma:
    image=cv2.imread(folder_path + '/glioma/' + image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((240,240))
    dataset.append(np.array(image))
    lab.append(1)

In [9]:
for image_name in meningioma:
    image=cv2.imread(folder_path + '/meningioma/' + image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((240,240))
    dataset.append(np.array(image))
    lab.append(2)

In [10]:
for image_name in pituitary:
    image=cv2.imread(folder_path + '/pituitary/' + image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((240,240))
    dataset.append(np.array(image))
    lab.append(3)

In [11]:
for image_name in craniopharyngioma:
    image=cv2.imread(folder_path + '/craniopharyngioma_final_1070/' + image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((240,240))
    dataset.append(np.array(image))
    lab.append(4)

In [12]:
dataset = np.array(dataset)
lab = np.array(lab)
print(dataset.shape, lab.shape)

(8103, 240, 240, 3) (8103,)


### Deep Feature Extraction

##### VGG16

In [13]:
from tensorflow.keras.applications import VGG16

base_model = tf.keras.applications.vgg16.VGG16(include_top =False,weights='imagenet')
base_model.trainable = False

from tensorflow.keras.layers import *
input = Input(shape=(240,240,3))
x = Rescaling(scale =1./127.5)(input)
x = base_model(x)
x = Flatten()(x)
x = Dense(units = 4096,activation='relu')(x)
x = Dense(units = 1000,activation = 'relu')(x)
output = Dense(5,activation = 'sigmoid')(x)
model = tf.keras.Model(input,output)

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics =['accuracy'])

In [14]:
from keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1./127.5)

subdirectories = ['no_tumor', 'glioma', 'meningioma', 'pituitary', 'craniopharyngioma_final_1070']

# Create a generator for the images in the subdirectories
generator = datagen.flow_from_directory(
    folder_path,
    target_size=(240,240),
    batch_size=32,
    class_mode='categorical',
#     class_mode=None,
    shuffle=False,
    classes=subdirectories)

Found 8103 images belonging to 5 classes.


In [15]:
vgg_features = model.predict(generator, verbose=1)

254/254 [==============================] - 419s 2s/step


In [16]:
print(len(vgg_features))

8103


In [17]:
print("Number of extracted deep features:", vgg_features.shape[1])

Number of extracted deep features: 5


In [18]:
np.save('extracted_features_vgg.npy', vgg_features)

In [19]:
vgg_features.shape

(8103, 5)

##### DenseNet

In [20]:
from tensorflow.keras.applications import DenseNet201

model = DenseNet201(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

from tensorflow.keras.applications.densenet import preprocess_input

dataset = preprocess_input(np.array(dataset))

In [21]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

subdirectories = ['no_tumor', 'glioma', 'meningioma', 'pituitary', 'craniopharyngioma_final_1070']

# Create a generator for the images in the subdirectories
generator = datagen.flow_from_directory(
    folder_path,
    target_size=(240,240),
    batch_size=32,
    class_mode='categorical',
#     class_mode=None,
    shuffle=False,
    classes=subdirectories)

Found 8103 images belonging to 5 classes.


In [22]:
dense_features = model.predict(generator, verbose=1)

254/254 [==============================] - 409s 2s/step


In [23]:
print(len(dense_features))

8103


In [24]:
print("Number of extracted deep features:", dense_features.shape[1])

Number of extracted deep features: 7


In [25]:
np.save('extracted_features_dense.npy', dense_features)

In [26]:
dense_features.shape

(8103, 7, 7, 1920)

In [27]:
# Reshape the features into a 2D array
num_samples = dense_features.shape[0]
reshaped_dense = dense_features.reshape(num_samples, -1)

##### InceptionResNetV2

In [29]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

dataset = preprocess_input(np.array(dataset))

In [30]:
from keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1./255)

subdirectories = ['no_tumor', 'glioma', 'meningioma', 'pituitary', 'craniopharyngioma_final_1070']

# Create a generator for the images in the subdirectories
generator = datagen.flow_from_directory(
    folder_path,
    target_size=(240,240),
    batch_size=32,
    class_mode='categorical',
#     class_mode=None,
    shuffle=False,
    classes=subdirectories)

Found 8103 images belonging to 5 classes.


In [31]:
inceptionResNet_features = model.predict(generator, verbose=1)

254/254 [==============================] - 298s 1s/step


In [32]:
print(len(inceptionResNet_features))

8103


In [33]:
print("Number of extracted deep features:", inceptionResNet_features.shape[1])

Number of extracted deep features: 6


In [34]:
np.save('inceptionResNet_features.npy', inceptionResNet_features)

In [35]:
inceptionResNet_features.shape

(8103, 6, 6, 1536)

In [36]:
# Reshape the features into a 2D array
num_samples = inceptionResNet_features.shape[0]
reshaped_inceptionResNet = inceptionResNet_features.reshape(num_samples, -1)

### Concatanation

In [38]:
# Input layer
from keras.layers import Concatenate, Input, Flatten, Dense

input_layer = Input(shape=(240, 240, 3))

In [40]:
# Concatenate the extracted features
concatenated_features = Concatenate()([reshaped_dense, vgg_features, reshaped_inceptionResNet])

In [45]:
print(len(concatenated_features))

8103


In [46]:
print(len(concatenated_features[1]))

149381


In [47]:
# save final features as 'concatenated_features' for future use
np.save("concatenated_features.npy", concatenated_features)

In [48]:
from tensorflow.keras.layers import Input

# Flatten the features to use them for classification
features_flatten = np.reshape(concatenated_features, (concatenated_features.shape[0], -1))

# Define the labels for the images in the subdirectories
labels = generator.classes

Apply the classification models afterwards. The model will generate results for the balanced dataset.